In [1]:
import os
import math
from scipy import integrate
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
# read of the geometry from a data file
naca_filepath = os.path.join('resources', 'naca0012.dat')
with open(naca_filepath,'r') as file_name:
    x, y = numpy.loadtext(file_name, dtype=float, delimiter='\t', unpack=True)
    
# plot the geometry
width = 10
plt.figure(figsize=(width, width))
plt.xlabel('x', fontsize=16)
plt.ylabel('y', fontsize=16)
plt.grid()
plt.plot(x, y, color='k', linestyle='-', linewidth=2)
plt.axis('scaled', adjustable='box')

FileNotFoundError: [Errno 2] No such file or directory: 'resources/naca0012.dat'

In [ ]:
class Panel:
    """
    Contains information related to a panel.
    """
    def __init__(self, xa, ya, xb, yb):
        """
        Initializes the panel.
        
        Sets the end-points and calculates the center, length,
        and angle (with the x-axis) of the panel.
        Defines if the panel is on the lower or upper surface of the geometry.
        Initializes the source-sheet strength, tangential velocity,
        and pressure coefficient to zero.
        
        Parameters
        ----------
        xa: float
            x-coordinate of the first end-point
        ya: float
            y-coordinate of the first end-point
        xb: float
            x-coordinate of the second end-point
        yb: float
            y-coordinate of the second end-point
        """
        self.xa, self.ya = xa, ya
        self.xb, self.yb = xb, yb
        
        self.xc, self.yc = (xa+xb)/2, (ya+yb)/2       # control(center) point of the panel
        self.length = ((xb-xa)**2+(yb-ya)**2)         # length of the panel
        
        # orientation of the panel (angle with x-axis)
        if xb-xa <=0.:
            self.beta = math.acos((yb-ya)/self.length)
        elif xb-xa > 0.:
            self.beta = math.pi + math.acos(-(yb-ya)/self.length)
            
        # location of the panel
        if self.beta <= math.pi:
            self.loc = 'upper'
        else:
            self.loc = 'lower'
            
        self.sigma = 0.                           # source strength
        self.vt = 0.                              # tangential velocity
        self.cp = 0.                              # coefficient of pressure

## Define the panels by creating a circle around the airfoil and marking the x-coordinates. The y-coordinates are found by projecting them from the circle onto the airfoil through interpolation. Panels are smaller near leading and trailing edges due to more curvature.

In [3]:
def define_panels(x, y, N=40):
    """
    Descretizes the geometry into panels using the 'cosine' method.
    
    Parameters
    ----------
    x: 1D array of floats
        x-coordinate of the points defining the geometry
    y: 1D array of floats
        y-coordinate of the points defining the geometry
    N: integer, optional
        Number of panels;
        default: 40.
        
    Returns
    -------
    panels: 1D numpy array of Panel objects
        the discretization of the geometry into panels
    """
    R = (x.max()-x.min())/2                                       # Radius of the circle
    x_center = (x.max()+x.min())/2                                # x-coord of the center
    x_circle = x_center + R*np.cos(np.linspace(0, 2*math.pi, N+1))  # x-coord of the circle points
    
    x_ends = np.copy(x_circle)                     # projection of the x-coord on the surface
    y_ends = np.empty_like(x_ends)                 # intiialization of the y-coord numpy array
    
    x, y = np.append(x, x[0]), np.append(y, y[0])  # extend arrays using numpy.append
    
    # computes the y-coordinate of the end-points
    I = 0
    for i in range(N):
        while I < len(x)-1:
            if (x(I) <= x_ends[i] <= x[I+1]) or (x[I+1] <= x_ends[i] <= x[I]):
                break
            else:
                I += 1
        a = (y[I+1] - y[I])/(x[I+1] - x[I])
        b = y[I+1] - a*x[I+1]
    y_ends[i] = y_ends[0]
    
    panels = np.empty(N, dtype = object)
    for i in range(N):
        panels[i] = Panel(x_ends[i], y_ends[i], x_ends[i+1], y_ends[i+1])
    
    return panels

In [4]:
N = 40                                 # number of panels
panels = define_panels(x, y, N)        # discretization of geometry into panels

# plot the geometry of the panels
width = 10
plt.figure(figsize=(width,width))
plt.grid()
plt.xlabel('x', fontsize = 16)
plt.ylabel('y', fontsize = 16)
plt.plot(x, y, color='k', linestyle='-', linewidth=2)
plt.plot(np.append([panel.xa for panel in panels], [panels[0].xa]),
        np.append([panel.ya for panel in panels], [panels[0].ya]),
        linestyle='-', linewidth=1, marker='o', markersize=6, color='#CD2305')
plt.plot(axis='scaled', adjustable='box')
plt.xlim(-0.1, 1.1)
plt.ylim(-0.1, 0.1);

NameError: name 'x' is not defined

## Create a Freestream class with angle of attack=0 and u_inf = 1.0

In [5]:
class Freestream:
    """
    Freestream conditions
    """
    def __init__(self, u_inf=1.0, alpha=0.0):
        """
        Sets the freestream speed and angle (with the x-axis)
        
        Parameters
        ----------
        u_inf: float, optional
            freestream speed;
            default: 1.0
        alpha: float, optional
            angle of attack in degrees;
            default: 0.0
        """
        self.u_inf = u_inf
        self.alpha = alpha*math.pi/180      # degrees to radians

In [6]:
# define and create the object freestream
u_inf = 1.0                                 # freestream speed
alpha = 0.0                                 # angle of attack
freestream = Freestream(u_inf, alpha)       # instantiation of the object freestream

## Generalized integral along each panel allowing for adaptation to direction of derivation

In [7]:
def integral(x, y, panel, dxdz, dydz):
    """
    Evaluates the contribution of a panel at one point.
    
    Parameters
    ----------
    x: float
        x-coordinate of the target point.
    y: float
        y-coordinate of the target point.
    panel: Panel object
        Source panel which contribution is evaluated.
    dxdz: float
        Derivative of x in the z-direction.
    dydz: float
        Derivative of y in the z-direction.
    
    Returns
    -------
    Integral over the panel of the influence at the given target point.
    """
    def integrand(s):
        return ( ((x - (panel.xa - math.sin(panel.beta)*s))*dxdz
                  +(y - (panel.ya + math.cos(panel.beta)*s))*dydz)
                / ((x - (panel.xa - math.sin(panel.beta)*s))**2
                   +(y - (panel.ya + math.cos(panel.beta)*s))**2) )
    return integrate.quad(integrand, 0.0, panel.length)[0]

## Build and solve a linear system of equations [A][$\sigma$]=[b]

In [8]:
def build_matrix(panels):
    """
    Builds the source matrix
    
    Parameters
    ----------
    panels: 1D array of panel objects
        The source panels.
    
    Returns
    -------
    A: 2D numpy array of floats
        The source matrix (NxN: N is the number of panels).
    """
    N = len(panels)
    A = np.empty((N,N), dtype=float)
    np.fill_diagonal(A, 0.5)
    
    for i, p_i in enumerate(panels):
        for j, p_j in enumerate(panels):
            if i != j:
                A[i,j] = 0.5/math.pi*integral(p_i.xc, p_i.yc, p_j, math.cos(p_i.beta), math.sin(p_i.beta))
                
    return A

def build_rhs(panels, freestream):
    """
    Builds the rhs of the linear system.
    
    Parameters
    ----------
    panels: 1D array of panel objects
        The source panels.
    freestream: Freestream object
        The freestream conditions.
    
    Returns
    -------
    b: 1D numpy array of floats
        RHS of the linear system.
    """
    b = np.empty(len(panels), dtype=float)
    
    for i, panel in enumerate(panels):
        b[i] = -freestream.u_inf * math.cos(freestream.alpha - panel.beta)
        
    return b

In [9]:
A = build_matrix(panels)                       # compute the singularity matrix
b = build_rhs(panels, freestream)              # compute the freestream RHS

NameError: name 'panels' is not defined

In [10]:
# solve the linear system to get sigma
sigma = np.linalg.solve(A,b)

for i, panel in enumerate(panels):
    panel.sigma = sigma[i]

NameError: name 'A' is not defined

In [11]:
def get_tangential_velocity(panels, freestream):
    """
    Computes the tangential velocity on the surface of the panel.
    
    Parameters
    ----------
    panels: 1D array of panel objects
        The source panels.
    freestream: Freestream object
        the freestream conditions.
    """
    N = len(panels)
    A = np.empty((N,N), dtype=float)
    np.fill_diagonal(A, 0.0)
    
    for i, p_i in enumerate(panels):
        for j, p_j in enumerate(panels):
            if i != j:
                A[i,j] = 0.5/math.pi*integral(p_i.xc, p_i.yc, p_j, -math.sin(p_i.beta), math.cos(p_i.beta))
    
    b = freestream.u_inf * np.sin([freestream.alpha - panel.beta for panel in panels])
    
    sigma = np.array([panel.sigma for panel in panels])
    
    vt = np.dot(A, sigma) + b
    
    for i, panel in enumerate(panels):
        panel.vt = vt[i]

In [12]:
# compute the tangential velocity at the center-point of each panel
get_tangential_velocity(panels, freestream)

NameError: name 'panels' is not defined